In [1]:
%run static_data.ipynb
%run solution.ipynb

In [2]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import os
import time

data = None
gg_map = None
gg_data = None

In [3]:
class GGMap:
    def __init__(self, sub_data):
        self.gg_map = sub_data
        self.direction_layers = sub_data['route_layer_api']
        self.mapping()
        
    def mapping(self):
        self.layers = self.gg_map['figure']._trait_values['_map'].layers[:]
        self.makers = self.layers[0]
        self.current_data_bounds = self.gg_map['figure']._trait_values['_map']._trait_values['data_bounds'][:]
        
    def retain_direction_layer_with_index(self, idx):
        select_direction_layer = self.direction_layers[idx]
        self.gg_map['figure']._trait_values['_map'].layers = [self.makers] 
        self.gg_map['figure'].add_layer(select_direction_layer)
#         self.gg_map['figure']._trait_values['_map']._trait_values['data_bounds'] = self.current_data_bounds
#         print(self.gg_map['figure']._trait_values['_map']._trait_values['data_bounds'])
    
    def restore_origin_state(self):
        time.sleep(1)
        self.gg_map['figure']._trait_values['_map'].layers = [self.makers] 
        for direction_layer in self.direction_layers[:]:
            self.gg_map['figure'].add_layer(direction_layer)
            time.sleep(0.2)
            
    def display_section(self, direction_layers):
        time.sleep(1)
        self.gg_map['figure']._trait_values['_map'].layers = [self.makers] 
        for direction_layer in direction_layers:
            self.gg_map['figure'].add_layer(direction_layer)
            time.sleep(0.2)
            
    def display_markers_only(self):
        time.sleep(1)
        self.gg_map['figure']._trait_values['_map'].layers = [self.makers] 

In [4]:
def handle_execute_btn(obj):
    global tour_btn, data, gg_data
    with output:
        if w_select:
            exe_btn.disabled = True
            data = execute(w_select.value[:-4], metric='duration', layout='100%')
            if data is not None:
                gg_data = GGMap(data)                
                total_route = len(data['route'])
                total_pages = total_route // 10 if (total_route % 10 == 0) else (total_route // 10) + 1
                tour_btn.options = ['All'] + ['Route Set {}'.format(i + 1) for i in range(total_pages)] + data['route']
                map_cont = widgets.Box([data['figure']], layout=widgets.Layout(height='auto', width='auto', align_items='stretch'))
                grid[:-2, 2:-2] = map_cont
            time.sleep(0.5)
    exe_btn.disabled = False
        
def handle_tour_select(change):
    global gg_data
    with output:
        if tour_btn.value == 'All':
            dir_btn.value = 'Please select specific route to see'
            if gg_data:
                gg_data.display_markers_only()
                map_cont = widgets.Box([data['figure']], layout=widgets.Layout(height='auto', width='auto', align_items='stretch'))
                grid[:-2, 2:-2] = map_cont
        elif 'Route' in tour_btn.value:
            dir_btn.value = 'Please select specific route to see'
            if gg_data:
                section_id = int(tour_btn.value[-1]) - 1
                section_range = range(section_id * 10, (section_id + 1) * 10 if len(data['route_layer_api']) >  (section_id + 1) * 10 else len(data['route_layer_api']))
                section_layers = [data['route_layer_api'][i] for i in section_range]
                gg_data.display_section(section_layers)
                map_cont = widgets.Box([data['figure']], layout=widgets.Layout(height='auto', width='auto', align_items='stretch'))
                grid[:-2, 2:-2] = map_cont
        else:
            desc_map = dict(zip(data['route'], data['route_description']))
            dir_btn.value = desc_map[tour_btn.value]
            if gg_data:
                gg_data.retain_direction_layer_with_index(int(tour_btn.value[:tour_btn.value.rfind(' ->')]))
                map_cont = widgets.Box([data['figure']], layout=widgets.Layout(height='auto', width='auto', align_items='stretch'))
                grid[:-2, 2:-2] = map_cont

In [5]:
w_select = widgets.Select(
    options=os.listdir('../data/tw_data/'),
    value=None,
    disabled=False, layout=widgets.Layout(height='100%', width='100%')
)

exe_btn = widgets.Button(description='Execute', button_style='success', layout=widgets.Layout(height='auto', width='auto'))
map_btn = widgets.Button(description='', button_style='', layout=widgets.Layout(height='auto', width='auto'))
map_cont = widgets.Box([map_btn], layout=widgets.Layout(height='100%', width='100%', align_items='stretch'))
tour_label = widgets.Label(value='Route from x to y (0 is depot)', layout=widgets.Layout(height='100%', width='100%'))
tour_btn = widgets.Select(options=[], value=None, description='', disabled=False, layout=widgets.Layout(height='100%', width='100%'))
dir_btn = widgets.HTML(value='', placeholder='Empty', disabled=True, layout=widgets.Layout(height='auto', width='auto'))
dir_cont = widgets.HBox([dir_btn], layout=widgets.Layout(height='100%', overflow_y='auto'))

In [6]:
grid = widgets.GridspecLayout(10, 16, height='900px')
grid[:-2, 2:-1] = map_cont
grid[1:, :2] = w_select
grid[0, :2] = exe_btn
grid[:1, -2:] = tour_label
grid[1:, -2:] = tour_btn
grid[-2:, 2:-2] = dir_cont

output = widgets.Output()
exe_btn.on_click(handle_execute_btn)
tour_btn.observe(handle_tour_select, names='value')

In [7]:
display(grid, output)

GridspecLayout(children=(Select(layout=Layout(grid_area='widget002', height='100%', width='100%'), options=('d…

Output()